## Can trading using RSI beat buying and holding a stock?

### Background: 
The Relative Strength Index (RSI) is a momentum indicator that describes the current price relative to average high and low prices over a previous trading period. This indicator estimates overbought or oversold status and helps spot trend reversals, price pullbacks, and the emergence of bullish or bearish markets.

The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, “New Concepts in Technical Trading Systems.”

An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.

Here is the process of calculating RSI:

1. For 14 periods, calculate the difference in price from the current period and the previous period;
2. For each period, record each positive change in price as a gain and each negative change as a loss;
3. On the 14th period, calculate the arithmetic mean of the gains and losses for the entire 14 day period (e.g. gains / 14 & losses / 14);
4. Use these values to calculate the RS
5. Use the RS value to calculate the RSI
6. For each proceeding period, use only the previous RSI value to calculate the next average value by multiplying by our lookback period – 1 (e.g. 13 for a lookback of 14).
7. Add the value obtained in step 6 to the current day’s value (do this for both the gains and losses)

<img src="rsi.jpeg" width=300 height=300 />

### Resource:

- https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209
- https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/
- https://en.wikipedia.org/wiki/Relative_strength_index

### Import Libraries and Finance Data 

In [1265]:
# import basic libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import altair as alt
from IPython.display import display_html
from itertools import chain,cycle

# adjust the width of jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

#three ways to import finance data, we will mainly use yfinance 
#https://pypi.org/project/yfinance    OR  https://www.alpharithms.com/python-financial-data-491110/

# import quandl
# df = quandl.get('WIKI/NVDA')

# import pandas_datareader as pdr
# #request API call from https://data.nasdaq.com/sign-up
# df = pdr.get_data_quandl("NVDA", api_key="vjsci8v8EfV-BpoQmxbR")

import yfinance as yf

C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1710285548.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1266]:
## Function to show dataframes side by side
## https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [1267]:
# Get finance data, valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
Ticker="SPY"
df = yf.Ticker(Ticker).history(period='3y')
df=df.reset_index()
df['Date']=pd.to_datetime(df['Date'])
df=df[['Date','Open','High','Low','Close']]
df['Close']=round(df['Close'],2)

In [1268]:
df.head()

,Date,Open,High,Low,Close
0,2019-04-08,274.265799,275.036902,273.570842,274.92
1,2019-04-09,273.904079,274.246779,272.933069,273.51
2,2019-04-10,273.951606,274.541859,273.513703,274.45
3,2019-04-11,274.960768,274.970297,273.770792,274.37
4,2019-04-12,276.074608,276.522041,274.418170,276.23


### Calculating RSI Value

My attempt on calculating the RSI value using simple average. But a typical RSI value is calculating using Wilder’s original smoothing method
<img src="wsm.jpeg" width=300 height=300 />

In [1269]:
#using the ta library, RSI values are calculated using daily close price. The default RSI period is 14 days.
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)

#Dropping the first 14 days without 14 days and reset index
df.drop(range(0,14),inplace=True)
df.reset_index(drop=True,inplace=True)

# Value is considered "overbought" when it exceeds 70 and "oversold" when indictator is below 30
df['Overbought']=70
df['Oversold']=30

In [1270]:
# uncomment if you want to see how RSI is calculated using Simple Averages

# df['Difference'] = df['Close'].diff()
# df['Difference'].fillna(0,inplace=True)
# df['PercentChange'] = round(df['Difference']/ df['Close']*100,2)
# df.loc[df['Difference']>0, 'Gain'] = df['Difference']
# df.loc[df['Difference']<0, 'Loss'] = -df['Difference']
# df['Gain'].fillna(0,inplace=True)
# df['Loss'].fillna(0,inplace=True)
# for i in range(0,df.shape[0]-14):
#     sumGain=0
#     sumLoss=0
#     for index, row in df[1+i:14+i].iterrows():
#         sumGain=sumGain+row['Gain']
#         sumLoss=sumLoss+row['Loss']
#     df.loc[df.index==(14+i),'AvgGain']=round(sumGain/8,2)
#     df.loc[df.index==(14+i),'SumGain']=sumGain    
#     df.loc[df.index==(14+i),'AvgLoss']=round(sumLoss/8,2)    
#     df.loc[df.index==(14+i),'SumLoss']=sumLoss    
# df['RS'] = round(df['AvgGain']/df['AvgLoss'],2)
# df['AvgGain'].fillna(0,inplace=True)
# df['AvgLoss'].fillna(0,inplace=True)
# df['RS'].fillna(0,inplace=True)
# df['RSI'] = round(100-(100/(1+df['RS'])),2)
# df=df.reset_index()
# df.drop(range(0,14),inplace=True)

In [1271]:
#Adding bolinger bands
# BBL is low
# BBM is m
# BBU is upper
df['BBUpper']= ta.bbands((df['Close']))['BBU_5_2.0']
df['BBmid']= ta.bbands((df['Close']))['BBM_5_2.0']
df['BBLower']= ta.bbands((df['Close']))['BBL_5_2.0']
df

,Date,Open,High,Low,Close,RSI,Overbought,Oversold,BBUpper,BBmid,BBLower
0,2019-04-29,279.416090,280.310955,279.320886,279.76,75.82,70,30,NaN,NaN,NaN
1,2019-04-30,279.397001,280.206191,277.902413,279.90,76.32,70,30,NaN,NaN,NaN
2,2019-05-01,280.567938,280.786904,277.788139,277.80,57.11,70,30,NaN,NaN,NaN
3,2019-05-02,277.673867,278.644906,275.617584,277.20,53.00,70,30,NaN,NaN,NaN
4,2019-05-03,278.759213,280.206214,277.312182,279.91,65.18,70,30,281.256430,278.914,276.571570
...,...,...,...,...,...,...,...,...,...,...,...
738,2022-03-31,457.890015,458.760010,451.160004,451.64,56.59,70,30,463.474627,456.098,448.721373
739,2022-04-01,453.309998,453.459991,449.140015,452.92,57.41,70,30,463.437445,456.144,448.850555
740,2022-04-04,453.130005,456.910004,452.260010,456.80,59.87,70,30,463.627345,456.322,449.016655
741,2022-04-05,455.220001,457.829987,449.820007,451.03,54.79,70,30,460.234596,454.218,448.201404


In [1272]:
#Prepare of Stock graph
Stock = alt.Chart(df).mark_line(color="blue").encode(
    x='Date:T',
    y='Close:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f'),
             alt.Tooltip('BBUpper:Q', format=',.2f'),
             alt.Tooltip('BBLower:Q', format=',.2f')]
).properties(
    title='Stock Price',
    width=800,
    height=300
)

BBU = alt.Chart(df).mark_line(color= 'green',size=1).encode(
    x='Date:T',
    y='BBUpper:Q'
)

#BBM= alt.Chart(df).mark_line(color= 'orange').encode(
#    x='Date:T',
#    y='BBmid:Q'
#)

BBL = alt.Chart(df).mark_line(color= 'red',size=1).encode(
    x='Date:T',
    y='BBLower:Q'
)

Stock_Chart=Stock+BBU+BBL

In [1273]:
Stock_Chart

alt.LayerChart(...)

In [1274]:
#Plotting RSI chart with Oversold and Overbought lines
RSI = alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='RSI:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    title='RSI',
    width=800,
    height=300)

Oversold = alt.Chart(df).mark_line(color= 'red').encode(
    x='Date:T',
    y='Oversold:Q'
)

Overbought = alt.Chart(df).mark_line(color= 'green').encode(
    x='Date:T',
    y='Overbought:Q'
)

RSI_Chart=RSI+Oversold+Overbought
Stock_Chart & RSI_Chart

alt.VConcatChart(...)

### Setting up RSI Strategy 

In [1275]:
print('Current Start Date :',df.iloc[0]['Date'])

Current Start Date : 2019-04-29 00:00:00


In [1276]:
# Set up your initial investment and time below
df = df.loc[df['Date']>='2019-04-15']
df.reset_index(drop=True,inplace=True)
df.head()

,Date,Open,High,Low,Close,RSI,Overbought,Oversold,BBUpper,BBmid,BBLower
0,2019-04-29,279.416090,280.310955,279.320886,279.76,75.82,70,30,NaN,NaN,NaN
1,2019-04-30,279.397001,280.206191,277.902413,279.90,76.32,70,30,NaN,NaN,NaN
2,2019-05-01,280.567938,280.786904,277.788139,277.80,57.11,70,30,NaN,NaN,NaN
3,2019-05-02,277.673867,278.644906,275.617584,277.20,53.00,70,30,NaN,NaN,NaN
4,2019-05-03,278.759213,280.206214,277.312182,279.91,65.18,70,30,281.25643,278.914,276.57157


In [1277]:
#initial values
startAmount=1000
startPrice=df.iloc[0]['Close']
shares=round(startAmount/startPrice,3)

In [1278]:
# choose your sell and buy RSI threshold, you would sell when RSI hits the sell threhold and rebuy when RSI drops below buy thresold
RSI_Sell=70
RSI_Buy=60

In [1279]:
#RSI Dateframe with Initialize Info
df_rsi = pd.DataFrame(columns = ['Date','Price','Shares', 'Amount','RSI','Buy/Sell','Profit/Loss','Profit/Loss %'])
df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'], 
              'Price' : startPrice,
              'Shares' : shares, 
              'Amount' : startAmount,
              'RSI': df.iloc[0]['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0},ignore_index=True)

df_longterm = df_rsi

print('Start Price:', '${:,.2f}'.format(startPrice))
print('Start Shares: ', shares)
print('RSI Sell Signal: >=', RSI_Sell)
print('RSI Buy Signal: <', RSI_Buy)
df_rsi.head()

Start Price: $279.76
Start Shares:  3.574
RSI Sell Signal: >= 70
RSI Buy Signal: < 60


C:\Users\nick\AppData\Local\Temp\ipykernel_30772\2588147253.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'],


,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2019-04-29 00:00:00,279.76,3.574,1000,75.82,Buy,0,0


In [1280]:
## buy and sell stock based on RSI value
Sell_Flag=0

principle=0
profit = 0
saveProfitFlag = 0 # 1: does not reinvest profit
additionalMoney = 1 # 1: invests more money with additionalMoneyTheshold is hit
additionalMoneyThreshold = 35
additionalMoneyValue = 1000
runningBuyTotal = 0
runningSharesTotal = 0

for index, row in df.iterrows():
    if index == 0:
        principle = df_rsi.iloc[-1]['Amount']
    
    

    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0: # SELLING
        print("In sell") 
        Sell_Flag=1
        if additionalMoney==1 and index!=1:
            print("Running Buy Total: "+str(runningBuyTotal))
            Sold_Amount=runningSharesTotal*row['Close']
            profit = profit + round(Sold_Amount-runningBuyTotal,2)
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : 0, 
                'Amount' : Sold_Amount,
                'RSI': row['RSI'],
                'Buy/Sell': 'Sell',
                'Profit/Loss': round(Sold_Amount-runningBuyTotal,2),
                'Profit/Loss %': round((Sold_Amount-runningBuyTotal)/runningBuyTotal*100,2)                              
                                },ignore_index=True)
        else:
            print("In reg sell")
            Sold_Amount=df_rsi.iloc[-1]['Shares']*row['Close']
            profit = profit + round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2)
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : 0, 
                'Amount' : Sold_Amount,
                'RSI': row['RSI'],
                'Buy/Sell': 'Sell',
                'Profit/Loss': round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2),
                'Profit/Loss %': round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                                },ignore_index=True)
        runningBuyTotal = 0
        runningSharesTotal = 0

    elif index!=0 and row['RSI']<=additionalMoneyThreshold and additionalMoney==1: # BUYING with additionalMoney
        Sell_Flag=0
        principle = principle + additionalMoneyValue
        runningBuyTotal = runningBuyTotal + additionalMoneyValue
        runningSharesTotal = runningSharesTotal + (additionalMoneyValue/row['Close'])
        df_rsi = df_rsi.append({
            'Date' : row['Date'], 
            'Price' : row['Close'],
            'Shares' : additionalMoneyValue/row['Close'], 
            'Amount' : additionalMoneyValue, #df_rsi.iloc[-1]['Amount'],
            'RSI': row['RSI'],
            'Buy/Sell': 'Buy+',
            'Profit/Loss':0,
            'Profit/Loss %':0                              
                            },ignore_index=True)
        print("buying extra")
    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1: # BUYING
        Sell_Flag=0
        if saveProfitFlag == 1 and df_rsi.iloc[-1]['Amount'] > principle:
            runningBuyTotal = runningBuyTotal + principle
            runningSharesTotal = runningSharesTotal + (principle/row['Close'])
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : principle/row['Close'], 
                'Amount' : principle, #df_rsi.iloc[-1]['Amount'],
                'RSI': row['RSI'],
                'Buy/Sell': 'Buy',
                'Profit/Loss':0,
                'Profit/Loss %':0                              
                                },ignore_index=True)
        else:
            runningBuyTotal = runningBuyTotal + df_rsi.iloc[-1]['Amount']
            runningSharesTotal = runningSharesTotal + (df_rsi.iloc[-1]['Amount']/row['Close'])
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : df_rsi.iloc[-1]['Amount']/row['Close'], 
                'Amount' : df_rsi.iloc[-1]['Amount'],
                'RSI': row['RSI'],
                'Buy/Sell': 'Buy',
                'Profit/Loss':0,
                'Profit/Loss %':0                              
                                },ignore_index=True)
        print("buying reg")
        
# TODO running... sell on the last day and calculate the final profit/loss        
if df_rsi.iloc[-1]['Buy/Sell']=='Buy' or 'Buy+':
    Sold_Amount=df.iloc[-1]['Close']*df_rsi.iloc[-1]['Shares']
    profit = profit + Sold_Amount-df_rsi.iloc[-1]['Amount']
    df_rsi = df_rsi.append({
              'Date' : df.iloc[-1]['Date'], 
              'Price' : df.iloc[-1]['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': df.iloc[-1]['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss':Sold_Amount-df_rsi.iloc[-1]['Amount'],
              'Profit/Loss %':round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
# add buy and sell threshold
df_rsi['Sell'] = RSI_Sell
df_rsi['Buy'] = RSI_Buy

C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.

In sell
In reg sell
buying reg
buying extra
buying extra
buying extra
buying extra
In sell
Running Buy Total: 5000.3626
buying reg
buying extra
In sell
Running Buy Total: 6361.725636994606
buying reg
In sell
Running Buy Total: 6666.214338070536
buying reg
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
In sell
Running Buy Total: 19834.260809698022
buying reg
In sell
Running Buy Total: 22556.836598115686


C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:87: FutureWarning: The frame.

buying reg
In sell
Running Buy Total: 25291.279497825624
buying reg
In sell
Running Buy Total: 28149.119032772804
buying reg
In sell
Running Buy Total: 30708.060618501448
buying reg
buying extra
buying extra
buying extra
In sell
Running Buy Total: 35150.56544059331
buying reg
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra
buying extra


C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_30772\1217252155.py:58: FutureWarning: The frame.

In [1281]:
# RSI BUT Buying and Selling when leaving overbought/sold zones

## buy and sell stock based on RSI value
#Sell_Flag=0
#principle=0
#profit = 0
#saveProfitFlag = 0 # 1: does not reinvest profit
#
#hitsOver = 0 # 1 when crosses over 70
#hitsUnder = 0 # 1 when crosses under 30
#
#for index, row in df.iterrows():
#    if index == 0:
#        principle = df_rsi.iloc[-1]['Amount']
#    
#    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0: # SELLING 
#        hitsOver = 1
#    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1: # BUYING    
#        hitsUnder = 1
#
#    if index!=0 and hitsOver==1 and Sell_Flag==0 and row['RSI']<RSI_Sell: #index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0: # SELLING 
#        Sell_Flag=1
#        hitsOver = 0
#        Sold_Amount=df_rsi.iloc[-1]['Shares']*row['Close']
#        profit = profit + round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2)
#        df_rsi = df_rsi.append({
#              'Date' : row['Date'], 
#              'Price' : row['Close'],
#              'Shares' : 0, 
#              'Amount' : Sold_Amount,
#              'RSI': row['RSI'],
#              'Buy/Sell': 'Sell',
#              'Profit/Loss': round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2),
#              'Profit/Loss %': round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
#                            },ignore_index=True)
#    
#    elif index!=0 and hitsUnder==1 and Sell_Flag==1 and row['RSI']>=RSI_Buy: #index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1: # BUYING
#        Sell_Flag=0
#        hitsUnder=0
#        if saveProfitFlag == 1 and df_rsi.iloc[-1]['Amount'] > principle:
#            df_rsi = df_rsi.append({
#                'Date' : row['Date'], 
#                'Price' : row['Close'],
#                'Shares' : principle/row['Close'], 
#                'Amount' : principle, #df_rsi.iloc[-1]['Amount'],
#                'RSI': row['RSI'],
#                'Buy/Sell': 'Buy',
#                'Profit/Loss':0,
#                'Profit/Loss %':0                              
#                                },ignore_index=True)
#        else:
#            df_rsi = df_rsi.append({
#                'Date' : row['Date'], 
#                'Price' : row['Close'],
#                'Shares' : df_rsi.iloc[-1]['Amount']/row['Close'], 
#                'Amount' : df_rsi.iloc[-1]['Amount'],
#                'RSI': row['RSI'],
#                'Buy/Sell': 'Buy',
#                'Profit/Loss':0,
#                'Profit/Loss %':0                              
#                                },ignore_index=True)
#        
#        
## sell on the last day and calculate the final profit/loss        
#if df_rsi.iloc[-1]['Buy/Sell']=='Buy':
#    Sold_Amount=df.iloc[-1]['Close']*df_rsi.iloc[-1]['Shares']
#    profit = profit + Sold_Amount-df_rsi.iloc[-1]['Amount']
#    df_rsi = df_rsi.append({
#              'Date' : df.iloc[-1]['Date'], 
#              'Price' : df.iloc[-1]['Close'],
#              'Shares' : 0, 
#              'Amount' : Sold_Amount,
#              'RSI': df.iloc[-1]['RSI'],
#              'Buy/Sell': 'Sell',
#              'Profit/Loss':Sold_Amount-df_rsi.iloc[-1]['Amount'],
#              'Profit/Loss %':round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
#                            },ignore_index=True)
#    
## add buy and sell threshold
#df_rsi['Sell'] = RSI_Sell
#df_rsi['Buy'] = RSI_Buy

In [1282]:
df_rsi

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-29 00:00:00,279.76,3.574,1000,75.82,Buy,0,0,70,60
1,2019-04-30 00:00:00,279.9,0,1000.3626,76.32,Sell,0.36,0.04,70,60
2,2019-05-01 00:00:00,277.8,3.601017,1000.3626,57.11,Buy,0,0,70,60
3,2019-05-13 00:00:00,267.37,3.740135,1000,28.03,Buy+,0,0,70,60
4,2019-05-29 00:00:00,264.91,3.774867,1000,33.47,Buy+,0,0,70,60
5,2019-05-31 00:00:00,262.05,3.816066,1000,30.46,Buy+,0,0,70,60
6,2019-06-03 00:00:00,261.39,3.825701,1000,29.62,Buy+,0,0,70,60
7,2019-07-03 00:00:00,285.84,0,5361.725637,71.55,Sell,361.36,7.23,70,60
8,2019-07-17 00:00:00,284.82,18.824962,5361.725637,59.97,Buy,0,0,70,60
9,2019-08-05 00:00:00,271.51,3.683106,1000,27.9,Buy+,0,0,70,60


In [1283]:
Final_RSI = alt.Chart(df_rsi).mark_circle(color='Blue',size=90).encode(
    x='Date:T',
    y='RSI:Q',
    color='Buy/Sell',
        tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    width=800,
    height=300
)

Sell_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5],color= 'red').encode(
    x='Date:T',
    y='Sell:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

Buy_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5], color= 'green').encode(
    x='Date:T',
    y='Buy:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

#RSI+Final_RSI+Sell_Line+Buy_Line

In [1284]:
df_longterm = df_rsi.iloc[[0, -1]]
df_longterm.reset_index(drop=True,inplace=True)
df_longterm.at[1, 'Amount'] = df_longterm.iloc[-1]['Price']*df_longterm.iloc[0]['Shares']
df_longterm.at[1, 'Buy/Sell'] = 'Sell'
df_longterm.at[1, 'Profit/Loss'] = df_longterm.iloc[-1]['Amount']-df_longterm.iloc[0]['Amount']
df_longterm.at[1, 'Profit/Loss %'] = round(df_longterm.at[1, 'Profit/Loss']/df_longterm.iloc[0]['Amount']*100,2)

In [1285]:
#Stock&(RSI+Final_RSI+Sell_Line+Buy_Line)
print("Profit: "+str(profit))
print("Principle: "+str(principle))
print("Percent gain: "+str(100*profit/principle))

Profit: 15820.725523423946
Principle: 31000
Percent gain: 51.03459846265789


In [1286]:
df_rsi.sum(axis = 0, skipna=True)

C:\Users\nick\AppData\Local\Temp\ipykernel_30772\3369551733.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_rsi.sum(axis = 0, skipna=True)


Price                                                     17553.06
Shares                                                  628.932515
Amount                                               425002.257858
RSI                                                        2281.44
Buy/Sell         BuySellBuyBuy+Buy+Buy+Buy+SellBuyBuy+SellBuySe...
Profit/Loss                                           15820.725523
Profit/Loss %                                                80.51
Sell                                                          3640
Buy                                                           3120
dtype: object

In [1287]:
display(df_rsi,df_longterm, titles=['RSI Strategy ','Long Term Buy&Hold'])

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-29 00:00:00,279.76,3.574,1000,75.82,Buy,0,0,70,60
1,2019-04-30 00:00:00,279.9,0,1000.3626,76.32,Sell,0.36,0.04,70,60
2,2019-05-01 00:00:00,277.8,3.601017,1000.3626,57.11,Buy,0,0,70,60
3,2019-05-13 00:00:00,267.37,3.740135,1000,28.03,Buy+,0,0,70,60
4,2019-05-29 00:00:00,264.91,3.774867,1000,33.47,Buy+,0,0,70,60
5,2019-05-31 00:00:00,262.05,3.816066,1000,30.46,Buy+,0,0,70,60
6,2019-06-03 00:00:00,261.39,3.825701,1000,29.62,Buy+,0,0,70,60
7,2019-07-03 00:00:00,285.84,0,5361.725637,71.55,Sell,361.36,7.23,70,60
8,2019-07-17 00:00:00,284.82,18.824962,5361.725637,59.97,Buy,0,0,70,60
9,2019-08-05 00:00:00,271.51,3.683106,1000,27.9,Buy+,0,0,70,60
